In [53]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from datetime import datetime, timedelta
import time
import os
import pandas as pd


#STORING ALL NEWS LINKS

page = 1
all_news_links = []


BASE_URL = "https://scroll.in/"
country = 'bangladesh'
initial_url = f'{BASE_URL}search?q={country}&page={page}'

#THE NEWS LISTS AREN'T RECOGNIZED BY BEAUTIFUL SOUP ALONE. THEY CAN BE RECOGNIZED WHEN PAGE IS LOADED USING SELENIUM FIRST,
#AND THEN BEAUTIFUL SOUP IS USED.

#SELENIUM CAUSES ADDITIONAL PROBLEMS WHILE USING 'NEXT' BUTTON. SO, ALL PAGES ARE SURFED MANUALLY. 

while page < 4:
    
    initial_url = f'{BASE_URL}search?q={country}&page={page}'
    
    driver = webdriver.Chrome()
    
    driver.get(initial_url)
    
    time.sleep(10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    news_lists = soup.find_all('li', {'itemscope':'itemscope'})

    for each_list in news_lists:

        all_news_links.append(each_list.find('a').get('href'))
        
    page+=1
    
    driver.quit()
        

#SCRAPING ALL NEWS LINKS

data_list = []
counter = 0

#USING SELENIUM, ALL LINKS HAVE TO BE LOADED BEFORE SCRAPING USING BEAUTIFUL SOUP. 

for url in all_news_links[:20]:

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        print(response.status_code)
        #TITLE

        title_tag = soup.find('meta', {'property':'og:title'})

        title = title_tag.get('content') if title_tag else 'Title not found'
        title_translation = 'None'


        #Content Summary

        content_summary_tag = soup.find('meta', {'property':'og:description'})
        content_summary = content_summary_tag.get('content') if content_summary_tag else 'Content Summary not found'
        summary_translation = 'None'

        #DATE INFO

        date_data = soup.find('meta',{'name':'dcterms.available'}).get('content')

        only_date = date_data.split('T')[0]
        only_time = date_data.split('T')[1]
        time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
        cleaned_date = f"{only_date},{time}"

        source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
        bangladesh_localtime = source_localtime + timedelta(minutes = 30)

        #AUTHOR

        author_tag = soup.find('meta',{'name':'dcterms.creator'})
        author = author_tag.get('content') if author_tag else 'Author not found'

        #CONTENT

        content = []

        main_content_div = soup.find('div', {'id':'article-contents'})

        if main_content_div:

            all_paras = main_content_div.find_all('p')

            for each_para in all_paras:
                content.append(each_para.text)

            full_content = ''.join(content)

        else:

            full_content = 'Content not found'

        #Some articles have 'main_content_div' but still produce blank in 'FUll CONTENT'
        #THUS, THE CODE BELOW

        if full_content == '':

            full_content = 'Content not found'
            
        content_translation = 'None'
        
        
        data_dict = {
        "url": url,
        "title": title,
        "content": full_content,
        "content_summary": content_summary,
        "title_translation":title_translation,
        "content_translation":content_translation,
        "summary translation":summary_translation,
        "author": author,
        "country": country,
        'source_localtime': source_localtime,
        'bangladesh_localtime': bangladesh_localtime

    }

        counter+=1


        if (full_content != "Content Not Found"):

                if data_dict not in data_list:
                    # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
                
                
df = pd.DataFrame(data_list)
df.head()


csv_filename = f"{country}_Scroll_IN.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


200
Link 1 added
200
Link 2 added
200
Link 3 added
200
Link 4 added
200
Link 5 added
200
Link 6 added
200
Link 7 added
200
Link 8 added
200
Link 9 added
200
Link 10 added
200
Link 11 added
200
Link 12 added
200
Link 13 added
200
Link 14 added
200
Link 15 added
200
Link 16 added
200
Link 17 added
200
Link 18 added
200
Link 19 added
200
Link 20 added


In [54]:
df = pd.DataFrame(data_list)
df.head()

,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://scroll.in/field/1063959/wpl-2024-meg-l...,WPL 2024: Meg Lanning and Co regroup as Delhi ...,Meg Lanning may have won the Orange Cap for th...,One of the more well-rounded squads in the WPL...,None,None,None,Scroll Staff,bangladesh,2024-02-22 14:00:00,2024-02-22 14:30:00
1,https://scroll.in/article/1063970/why-assams-m...,Why Assam’s Muslims see discrimination in stat...,"In August last year, Md Ruhul Amin and Shaiful...",The Assam chief minister recently declared tha...,None,None,None,Rokibuz Zaman,bangladesh,2024-02-22 06:30:00,2024-02-22 07:00:00
2,https://scroll.in/article/1063967/how-one-man-...,How one man is spreading Bangla education amon...,Morning sunlight on the misty valleys turned t...,Maijesh Tripura believes that exposure to Bang...,None,None,None,Faisal Mahmud,bangladesh,2024-02-21 13:30:00,2024-02-21 14:00:00
3,https://scroll.in/field/1064008/wpl-2024-forma...,"WPL 2024: Format, schedule, squads – All you n...",The second edition of the Women’s Premier Leag...,With the T20 World Cup scheduled for later thi...,None,None,None,Scroll Staff,bangladesh,2024-02-21 08:30:00,2024-02-21 09:00:00
4,https://scroll.in/latest/1064001/aadhaar-cards...,Aadhaar cards being deactivated in West Bengal...,West Bengal Chief Minister Mamata Banerjee on ...,The Centre has said that the deactivations occ...,None,None,None,Scroll Staff,bangladesh,2024-02-20 13:28:00,2024-02-20 13:58:00
